In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_4122/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_4122/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_7_5,0.964036,0.763130,0.886152,0.957823,0.016453,0.140616,0.096969,0.014092,0.068882,0.128268,0.925753,0.130289,74.214519,145.969299,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_43_7,0.968960,0.762229,0.893402,0.947632,0.014200,0.141151,0.090794,0.016550,0.063762,0.119166,0.935917,0.121043,74.508957,146.263737,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_8_2,0.965790,0.761733,0.888003,0.975549,0.015651,0.141446,0.095392,0.009979,0.067318,0.125102,0.929373,0.127073,74.314491,146.069270,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_44_2,0.969090,0.761715,0.892446,0.977646,0.014141,0.141456,0.091608,0.007068,0.063649,0.118915,0.936187,0.120787,74.517401,146.272181,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_7_3,0.963275,0.760104,0.881944,0.958264,0.016801,0.142413,0.100553,0.013944,0.069458,0.129618,0.924182,0.131660,74.172641,145.927421,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,model_4_0_0,-0.018928,-0.025043,0.150370,-0.001160,0.466144,0.608510,0.723665,0.793686,0.298199,0.682748,2.976103,0.693500,195.526519,406.442084,"Hidden Size=[24], regularizer=0.02, learning_r..."
674,model_2_30_3,0.899368,-0.031618,0.455390,0.959165,0.046038,0.612413,0.463866,0.015821,0.086305,0.214564,0.792243,0.217943,72.156586,143.911366,"Hidden Size=[8], regularizer=0.02, learning_ra..."
675,model_1_0_0,0.126061,-0.038606,0.149779,0.279332,0.399814,0.616561,0.724168,0.367519,0.373726,0.632309,-0.190045,0.642267,35.833510,72.798094,"Hidden Size=[4], regularizer=0.02, learning_ra..."
676,model_2_30_0,0.850005,-0.252752,0.472011,0.965755,0.068620,0.743688,0.449710,0.013268,0.092532,0.261955,0.690334,0.266080,71.358333,143.113113,"Hidden Size=[8], regularizer=0.02, learning_ra..."
